In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import pandas as pd
import numpy as np
from scipy.linalg import svd
from scipy.sparse.linalg import svds


lounge_df = pd.read_csv('/content/drive/My Drive/데이터톤/보고서 작성 시 사용할 최종 파일/데이터 및 전처리/라운지+혼잡도/라운지 데이터 최종.csv',encoding='euc-kr')
lounge_df['건물'] = lounge_df['건물'].replace({' ':''}, regex=True)
building_distance = pd.read_csv('/content/drive/My Drive/데이터톤/보고서 작성 시 사용할 최종 파일/데이터 및 전처리/건물별 이동시간 최종.csv',encoding='utf-8')

con_0 = pd.read_csv('/content/drive/My Drive/데이터톤/보고서 작성 시 사용할 최종 파일/데이터 및 전처리/라운지+혼잡도/혼잡도_월.csv')
con_1 = pd.read_csv('/content/drive/My Drive/데이터톤/보고서 작성 시 사용할 최종 파일/데이터 및 전처리/라운지+혼잡도/혼잡도_화.csv')
con_2 = pd.read_csv('/content/drive/My Drive/데이터톤/보고서 작성 시 사용할 최종 파일/데이터 및 전처리/라운지+혼잡도/혼잡도_수.csv')
con_3 = pd.read_csv('/content/drive/My Drive/데이터톤/보고서 작성 시 사용할 최종 파일/데이터 및 전처리/라운지+혼잡도/혼잡도_목.csv')
con_4 = pd.read_csv('/content/drive/My Drive/데이터톤/보고서 작성 시 사용할 최종 파일/데이터 및 전처리/라운지+혼잡도/혼잡도_금.csv')


#############   사용자의 라운지 점수 계산    #########################
######################################################################
def true_label(theme,location,time,num_user,table,plug,computer,facility):  
    user_building = location
    distance = building_distance[['building', user_building]].copy()
    distance.rename(columns = {user_building: '거리'}, inplace = True)

    #라운지	위치(층)	책상 높이	테이블 수	의자 수	냉난방장치	콘센트 수	창문	컴퓨터	정수기	화장실	혼잡도	건물	***거리
    lounge_with_distance = pd.merge(lounge_df, distance, left_on= '건물', right_on='building', how ='left' ).drop(columns=['building'])

    day = str(pd.to_datetime(time.split('T')[0]).dayofweek)
    user_c = con_0

    #find day
    if day==1:
        user_c = con_1
    elif day==2:
        user_c = con_2
    elif day==3:
        user_c = con_3
    elif day==4:
        user_c = con_4

    # 사용자 시간 인식
    range = pd.date_range("09:00", "23:30", freq="30min").strftime('%H:%M:%S')
    dt = pd.to_datetime(time).round('30min')
    user_c_r = range.get_loc(str(dt).split(' ')[1].split('+')[0])

    # 라운지	위치(층)	책상 높이	테이블 수	의자 수	냉난방장치	콘센트 수	창문	컴퓨터	정수기	화장실	***혼잡도	건물    거리
    final_lounge = lounge_with_distance
    final_lounge['혼잡도']=final_lounge.apply(lambda row: user_c[row.건물][user_c_r], axis=1)

    
    # 정규화
    '''
    norm lists :-
        1) desk_norm : 책상높이가 사용자 선호와 같으면 1
        2) facilities_norm : 정수기/화장실이 사용자 선호와 가까울수록 1 (하나당 -0.5)
        3) chair_norm : 의자 수 >= 사용자 수 면 1
        4) person_per_table_norm : 사용자 수와 책상별 앉을 수 있는 사람의 수가 비슷할수록 1
        5) plugs_norm : 콘센트 수 ~ 사용자 필요도
        6) computers_norm : 컴퓨터 수 ~ 사용자 필요도
        7) distance_norm : 거리가 멀수록 0 가까울수록 1
        8) congestion_norm : 혼잡할수록 0 가까울수록 1
        9) ac_norm : 있으면 1 없으면 0
        10) window_norm : 있으면 1 없으면 0
    '''
    satisfy_user_idx = final_lounge['책상 높이'] == table
    desk_norm = final_lounge['라운지'].copy()
    desk_norm[satisfy_user_idx] = 1
    desk_norm[~satisfy_user_idx] = 0

    satisfy_user_idx = final_lounge['정수기'] == facility
    facilities_norm = final_lounge['라운지'].copy()
    facilities_norm[satisfy_user_idx] = 0.5
    facilities_norm[~satisfy_user_idx] = 0.0
    satisfy_user_idx = final_lounge['화장실'] == facility
    facilities_norm[satisfy_user_idx] += 0.5
    facilities_norm[~satisfy_user_idx] += 0.0

    satisfy_user_idx = final_lounge['의자 수'] >= num_user
    chair_norm = final_lounge['라운지'].copy()
    chair_norm[satisfy_user_idx] = 1
    chair_norm[~satisfy_user_idx] = 0

    person_per_table_diff = abs(num_user - (final_lounge['의자 수']/final_lounge['테이블 수']))
    person_per_table_norm = 1 - (person_per_table_diff/(person_per_table_diff.max()-person_per_table_diff.min()))

    plugs = (final_lounge['콘센트 수']/final_lounge['의자 수'])*plug
    plugs_norm = (plugs-plugs.min())/(plugs.max()-plugs.min())

    #멀수록 0 가까울수록 1
    distance_norm = 1 - (final_lounge['거리']-final_lounge['거리'].min())/(final_lounge['거리'].max()-final_lounge['거리'].min())

    #혼잡할수록 0 한가할수록 1
    congestion_norm = 1 - (final_lounge['혼잡도']-final_lounge['혼잡도'].min())/(final_lounge['혼잡도'].max()-final_lounge['혼잡도'].min())

    computers = (final_lounge['컴퓨터']/final_lounge['의자 수'])*computer
    computers_norm = (computers-computers.min())/(computers.max()-computers.min())

    ac_norm = final_lounge['냉난방장치']

    window_norm = final_lounge['창문']

    # 테마 구별
    # 1. 휴식
    if theme == 1:
        scores = desk_norm*0.71+facilities_norm*0.68+chair_norm+person_per_table_norm+plugs_norm*0.68+computers_norm*0.115+distance_norm*4+congestion_norm*5+ac_norm*0.5+window_norm*0.1
    # 2. 대화
    elif theme == 2:
        scores = desk_norm*0.15+facilities_norm*0.68+chair_norm+person_per_table_norm+plugs_norm*0.88+computers_norm*0.115+distance_norm*4+(1-congestion_norm)*3.5+ac_norm*0.5+window_norm*0.1
    # 3. 팀공부
    elif theme == 3:
        scores = desk_norm*0.71+facilities_norm*0.68+chair_norm+person_per_table_norm+plugs_norm*0.88+computers_norm*0.115+distance_norm*4+(1-congestion_norm)*3.5+ac_norm*0.5+window_norm*0.1
    # 0. 공부
    else:
        scores = desk_norm*0.71+facilities_norm*0.68+chair_norm+person_per_table_norm+plugs_norm*0.68+computers_norm*0.115+distance_norm*4+congestion_norm*3+ac_norm*0.5+window_norm*0.1
    return scores
#############################################################################




###########  사용자에 라운지 9개 추천 #############################
###################################################################
def lounge_Top9(theme,location,time,num_user,table,plug,computer,facility):
  input_user_score = true_label(theme,location,time,num_user,table,plug,computer,facility)

  score_df = pd.read_csv('/content/drive/My Drive/데이터톤/보고서 작성 시 사용할 최종 파일/데이터 및 전처리/라운지+혼잡도/라운지_트루라벨_0(default).csv',encoding='euc-kr')
  if theme ==1:
        score_df = pd.read_csv('/content/drive/My Drive/데이터톤/보고서 작성 시 사용할 최종 파일/데이터 및 전처리/라운지+혼잡도/라운지_트루라벨_1.csv',encoding='euc-kr')
  elif theme ==2:
        score_df = pd.read_csv('/content/drive/My Drive/데이터톤/보고서 작성 시 사용할 최종 파일/데이터 및 전처리/라운지+혼잡도/라운지_트루라벨_2.csv',encoding='euc-kr')
  elif theme==3:
        score_df = pd.read_csv('/content/drive/My Drive/데이터톤/보고서 작성 시 사용할 최종 파일/데이터 및 전처리/라운지+혼잡도/라운지_트루라벨_3.csv',encoding='euc-kr')


  score_df.score = score_df.score.apply(eval)
  score_df.score = score_df.score.apply(np.array)

  # matrix로 변환
  user_score_matrix = np.array(list(score_df['score']))
  user_score_matrix = np.concatenate((user_score_matrix, np.array(input_user_score).reshape((1,229))), axis=0)   # 229 == 총 라운지 개수

  # 평균을 이용한 유저별 중심화
  user_score_mean = np.mean(user_score_matrix, axis=1)
  user_score_matrix_centered = user_score_matrix - user_score_mean.reshape(-1, 1)
  user_score_matrix_centered = user_score_matrix_centered.astype(np.float64)

  # Singular Value Decomposition
  # k는 latent factor의 개수라고 생각하면 됨. 개수 조정하면 다른 결과 얻을 수 있음.
  # U, s, Vt = svd(user_score_matrix_centered, full_matrices=False)
  U, s, Vt = svds(user_score_matrix_centered, k=3)
  S = np.diag(s)

  # SVD로 각 사용자별 점수 예측
  predicted_scores = np.dot(np.dot(U, S), Vt) + user_score_mean.reshape(-1, 1)
  predicted_scores[0]

  # 예측 행렬에서 사용자 index에 대해 라운지 index 9개 추천해주는 함수
  def make_recommendation(original, svd_prediction, user_index, num_recommendations=9):
    score = svd_prediction[user_index]
    rec_inx = score.argsort()[-num_recommendations:][::-1]
    rec_val = svd_prediction[user_index][rec_inx]
    return rec_inx, rec_val

  # rec_inx가 라운지 index
  which_user = -1
  rec_inx, rec_val = make_recommendation(user_score_matrix_centered, predicted_scores, which_user, 9)
  rec_inx

  # 최종 추천 라운지 9개 결정
  top9_df = pd.DataFrame(rec_inx, columns =['lounge_index'])
  top9 = pd.merge(top9_df, lounge_df, on= 'lounge_index', how ='left')

  output = top9[['건물', '라운지']].copy().to_records(index=False)

  return output
##############################################################


# 챗봇 input 넣는 곳
result = lounge_Top9(0, 'L-P', '2019-08-30T11:02:08+00:00',3,True,4,2,False)
print(result)

[('현대자동차경영관', '현대자동차경영관-3') ('현대자동차경영관', '현대자동차경영관-6')
 ('현대자동차경영관', '현대자동차경영관-2') ('동원글로벌리더십홀', '동원글로벌리더십홀-3')
 ('법학관(구관)', '법학관(구관)-4') ('L-P', 'L-P-1') ('운초우선교육관', '운초우선 교육관-3')
 ('운초우선교육관', '운초우선 교육관-4') ('L-P', 'L-P-3')]
